In [1]:
# Second generation video processing:
## List of imports (keep it simple)
import numpy as np
import scipy 
from scipy.integrate import odeint
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import subprocess
import tensorflow as tf
from PIL import Image,ImageDraw
import sys

/Users/taigaabe/anaconda3/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## In order to set up numerical integration, we need a callable function that calculates derivatives of the movement
## as a set of first order differential equations. Thus we define the following

## Define a form on forces as an external effect (we need a form that will return a value for non-integer evaluated 
## points in t):

def ball(y,t,env_params,dyn_params,ball_params,F):
    
    # Define Effect of forces
    # We have the forces determined as Fx and Fy, a sequence that the input gets to see
    # Determine effect of forces:
    F_vec = np.zeros((4))
    F_vec[3] -= dyn_params['grav_constant']*ball_params['mass'] ## Effect of gravity
    F_vec[2:4] += F(t) ## Deterministic exterior force being applied
    
    ## We want elasticity: 
    if np.any(abs(y[0:2])>env_params['movement_range']):
        F_imp = dyn_params['elasticity']*(abs(y[0:2])>env_params['movement_range'])*100000 ## Bounce back
        F_elast = F_imp*-np.sign(y[0:2])
  
    else:
        F_elast = np.zeros((2))
    F_vec[2:4] += F_elast
    
    ## F = ma
    a_vec = F_vec/ball_params['mass']
    
    ## Write down the full dynamics equation:
    ydot = np.dot(y,dyn_params['dyn_matrix'])+a_vec

    return ydot

## Turn the above into a function. 
## Inputs:
# imsize: width or height of image
# nb_w: number of plane waves to use per color channel (rgb)
# nb_knobs: dimensionality for the low-d parametrization of images
# nb_knobset: number of samples to take of those parameter values (sampled random uniform)
# alpha: overall scaling factor on sampled parameters (knobs)
# beta: additive constant for sampled parameters (knobs)

## Outputs:
## images: (nb_knobset images of shape(imsize,imsize,3))
## knobs: the settings of parameters that generated these images.
def imgen(imsize=64,nb_w = 50,nb_knobs = 2,nb_knobset = 30,alpha = 5,beta = 0,knobs = None,loading = None):
    ## Define xx and yy grids that will make functions easier to define
    x = np.linspace(0,10,imsize)
    y = np.linspace(0,10,imsize)

    xx,yy = np.meshgrid(x,y)
    
    ## Define lambda function that takes you from parameter space into plane waves:
    
    ## This is like sampling points randomly in 2-d fourier space, and then looking at the corresponding images. 
    
    ## Each element of the fourier series that we consier is a "plane wave" that is characterized by four parameters. 
    ## These parameters are: 
    ## 1. Amplitude: given a hard limit of 0,1, we rescale all amplitudes to be centered on 0.5 with corresponding baseline 
    ## amplitude, the parameter here is a scale that varies plane wave amplitudes relative to one another.
    ## 2. Phase: this parameter just describes the shift of the plane wave phase relative to some baseline. 
    ## 3. Period: multiply by some factor of two pi. otherwise unconstrained. 
    ## 4. Angle: a scalar value that determines the degree to which the plane wave varies in the x and y direction. the angle 
    ## should be transformed as sin and cosine to ensure uniformity in some sense 
    
    wave = lambda params: params[0]*np.sin(params[2]*(np.sin(params[3])*xx+np.cos(params[3])*yy+params[1]+0.5))

    ## Define a loading matrix to transform our low-d thing into parameters for a set of waves
    if loading is not None:
        loading_matrix = loading
    else:
        loading_matrix = np.random.uniform(size = (4,nb_knobs,3*nb_w))
    
    ## If the values of the parameters are given, use those. If not, generate from uniform (scaled appropriately)
    if knobs is not None:
        params = knobs
    else: 
        params = alpha*np.random.uniform(size = (nb_knobset,nb_knobs))+beta#np.linspace(0,10,nb_knobset*nb_knobs).reshape(nb_knobset,nb_knobs)
    
    params_transformed = np.dot(params,loading_matrix)
    
    ## Now iterate through plane waves, collapse into channels, normalize and collapse into images.
    allind = range(nb_knobset)
    all_images = []
    for i in allind:
        relevant_params = params_transformed[i,:,:]
        grids = np.array([wave(relevant_params[:,i]) for i in range(3*nb_w)])
        ## Now split into channels:
        channelcomponents = np.split(grids,3,axis = 0)
        ## Now collapse each:
        channels = [np.sum(channelcomp,axis = 0)/float(nb_w) for channelcomp in channelcomponents]
        ## Now normalize each:
        normchannels = [(channel-np.min(channel))/(np.max(channel)-np.min(channel)) for channel in channels]
        ## combine into an image:
        image = np.stack(normchannels,axis = -1)
        all_images.append(image)
    return all_images,params,loading_matrix

## Designate helper function to define features for examples more easily
def _int64_feature_(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature_(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _float_feature_(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))





In [3]:
## List of parameters
m = 10.0 # mass of ball
r = 6 # radius of ball

ball_params = {'mass':m,'radius':r}

dim = 2 #dimensionality of the problem
g = 0 #m/(s^2) (gravitational constant)
mv_range = 26 # given as a radius
d = mv_range+ball_params['radius'] # size of a square enclosure
t = 500 # time to simulate for ('seconds')
dt = 5000 # steps per second
# w = 20
# h = 20
dpi = 4
nb_backgrounds = 10
env_params = {'dimension':dim,
              'grav_constant':g,
              'enclosure_size':d,
              'movement_range':mv_range,
              'simtime':t,
              'timesteps':dt*t,
              'dpi':dpi}


A = np.zeros((4,4))
A[2,0] = 1
A[3,1] = 1 ## dynamics matrix
e = 1 ## reduction in force 
dyn_params = {'grav_constant':g,'dyn_matrix':A,'elasticity':e}



In [4]:
backgrounds,params,loadmat = imgen(imsize = 2*env_params['enclosure_size'],nb_knobset = nb_backgrounds)

In [5]:
## Solve for given F,y0,t
F = lambda t: np.array([-5*np.sin(t),5*np.cos(t/5.)-1])
# F = lambda t: np.array([10*np.sin(t/5.),10*np.cos(t/6.)])
# F = lambda t: np.array([0,0])
y0 = np.zeros((4))
y0[2]-= (1./3)/10
y0[0]-=20
t = np.linspace(0,nb_backgrounds*env_params['simtime'],env_params['timesteps'])
sol = odeint(ball, y0, t, args=(env_params,dyn_params,ball_params,F))

KeyboardInterrupt: 

In [ ]:
fig,ax = plt.subplots(3,1,figsize = (5,15))
ax[0].plot(sol[:,0],sol[:,1])
ax[0].set_title('Trial Trajectory for Ball')
ax[0].set_xlabel('X position')
ax[0].set_ylabel('Y position')
ax[1].plot(t,sol[:,0])
ax[1].set_title('X Trajectory for Ball')
ax[1].set_xlabel('time')
ax[1].set_ylabel('X position')
ax[2].plot(t,sol[:,1])
ax[2].set_title('Y Trajectory for Ball')
ax[2].set_xlabel('time')
ax[2].set_ylabel('Y position')
plt.tight_layout()
plt.show()

In [ ]:
all_datadirect = 'datadirectory'
try:
    os.mkdir(all_datadirect)
except:
    print('all data directory already exists!')

videodirectory = 'toydynamics_nograv'
try:
    os.mkdir(all_datadirect+'/'+videodirectory)
except:
    print('Directory already exists!')
# Sample at the seconds timescale:
frames = np.linspace(0,env_params['timesteps']-1,nb_backgrounds*env_params['simtime']).astype('int')
print('Generating Frames')

# Preprocess 
# sol[sol>(env_params['movement_range']-0.5)]=env_params['movement_range']-0.5
# sol[sol<(-env_params['movement_range']+0.5)]=-env_params['movement_range']+0.5
background = np.zeros((2*env_params['enclosure_size'],2*env_params['enclosure_size'],3)).astype(np.uint8)




## Now simulate and write:

datafile_name = "colorback_real.tfrecords"


writer = tf.python_io.TFRecordWriter(datafile_name)
for i,ti in enumerate(frames):
    j = i/500
    bias = env_params['enclosure_size']
    img = Image.fromarray(((255*backgrounds[j]).astype(np.uint8)))  
    draw = ImageDraw.Draw(img)
    print(sol[ti,0])
    draw.ellipse((sol[ti,0]-ball_params['radius']+bias, sol[ti,1]-ball_params['radius']+bias, sol[ti,0]+ball_params['radius']+bias, sol[ti,1]+ball_params['radius']+bias), fill=(255,255,255,255))# Create a PIL image
    img.save(all_datadirect+'/'+videodirectory+'/'+'colorframe%04d.png'%i)
    frame = np.array(img.getdata()).reshape(img.size[0], img.size[1], 3)/255.
    print(frame[32,32,0])
    example = _encoder_example_as_writing(datafile_name,frame,i,params[j,:],sol[ti,:])
    writer.write(example.SerializeToString())
writer.close()
sys.stdout.flush()

In [ ]:
os.chdir(all_datadirect+'/'+videodirectory)
print("Generating video")

video_title = 'blackback'
    
subprocess.call(['ffmpeg', '-framerate', str(30), '-i', 'colorframe%04d.png', '-r', '30',video_title+'.mp4'])



In [ ]:
os.chdir('../../')

In [ ]:
os.getcwd()

In [ ]:
np.save(video_title+'loadmat',loadmat)
np.save(video_title+'params',params)
np.save(video_title+'pos',sol)

In [ ]:
sol